In [1]:
# !pip install scikeras
# !pip install TensorFlow

In [2]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
# from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn import tree
from six import StringIO  
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as pyplot
from sklearn.metrics import roc_auc_score, roc_curve
# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from warnings import simplefilter
import time
start = time.time()

#with two dataset splitted
dftrain = pd.read_csv("train70_reduced.csv") 
dftest = pd.read_csv("test30_reduced.csv")
simplefilter(action='ignore', category=FutureWarning)
seed = 7

'''
#one dataset to be splitted
df = pd.read_csv("mqttdataset.csv") 
seed = 7
class_names = df.target.unique()
df=df.astype('category')
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

x_columns = df.columns.drop('target')
x = df[x_columns].values
y = df['target']

print("Ready to generate train and test datasets")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=seed)
print("x_train, y_train, x_test, y_test" + str(x_train.shape) + "" +str(y_train.shape) + "" +str(x_test.shape) + "" +str(y_test.shape))
'''

#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']


print("Ready to generate train and test datasets")

#Neural network
print("Starting Random forest")
model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(30, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(20, kernel_initializer='normal'))
model.add(Dense(6,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=200,batch_size=1000) 
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_nn = model.predict(x_test)
y_pred_nn = np.argmax(y_pred_nn,axis=1)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

print(model.summary())


#RandomForest
print("Starting Random forest")
classifier = RandomForestClassifier(verbose=2,random_state=seed)
classifier.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_random = classifier.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Create Naive Bayes Classifier
print("Starting Naive Bayes")
gnb = GaussianNB()
gnb.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_nb = gnb.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Decision tree
print("Starting Decision tree")
clf = DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_dt = clf.predict(x_test)
y_pred_dt_roc = clf.predict_proba(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Multi layer perceptron
print("Starting Multi layer perceptron")
model = MLPClassifier( max_iter=130, batch_size=1000, alpha=1e-4, activation = 'relu',solver='adam', verbose=10, tol=1e-4, random_state=seed)
model.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_mlp = model.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Gradient boost
print("Starting Gradient boost")
model = GradientBoostingClassifier(n_estimators=20, random_state=seed,verbose=2)
model.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_gradient = model.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))


print("Decision Tree, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_dt)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_dt,average='weighted')))
matrixdt = confusion_matrix(y_test,y_pred_dt)
print(matrixdt)


print("Naive Bayes, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_nb)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_nb,average='weighted')))
matrixnv = confusion_matrix(y_test,y_pred_nb)
print(matrixnv)


print("Neural network, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_nn)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_nn,average='weighted')))
matrixnn = confusion_matrix(y_test,y_pred_nn)
print(matrixnn)

print("MultiLayerPerceptron, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_mlp)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_mlp,average='weighted')))
matrixml = confusion_matrix(y_test,y_pred_mlp)
print(matrixml)
print("Random Forest, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_random)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_random,average='weighted')))
matrixrf = confusion_matrix(y_test,y_pred_random)
print(matrixrf)
print("GradienBoost, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_gradient)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_gradient,average='weighted')))
matrixgb = confusion_matrix(y_test,y_pred_gradient)
print(matrixgb)


2024-01-30 04:15:18.421930: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 04:15:18.572053: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 04:15:18.572174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 04:15:18.575381: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 04:15:18.594578: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 04:15:18.595914: I tensorflow/core/platform/cpu_feature_guard.cc:1

Ready to generate train and test datasets
Starting Random forest


2024-01-30 04:15:47.279878: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/200
232/232 - 3s - loss: 0.9414 - accuracy: 0.7656 - val_loss: 0.6330 - val_accuracy: 0.7799 - 3s/epoch - 14ms/step
Epoch 2/200
232/232 - 1s - loss: 0.5293 - accuracy: 0.7868 - val_loss: 0.5115 - val_accuracy: 0.7894 - 1s/epoch - 6ms/step
Epoch 3/200
232/232 - 1s - loss: 0.4509 - accuracy: 0.8115 - val_loss: 0.4705 - val_accuracy: 0.8222 - 1s/epoch - 6ms/step
Epoch 4/200
232/232 - 1s - loss: 0.4238 - accuracy: 0.8540 - val_loss: 0.4275 - val_accuracy: 0.8815 - 1s/epoch - 6ms/step
Epoch 5/200
232/232 - 1s - loss: 0.3663 - accuracy: 0.8844 - val_loss: 0.3956 - val_accuracy: 0.8572 - 1s/epoch - 6ms/step
Epoch 6/200
232/232 - 1s - loss: 0.3283 - accuracy: 0.8905 - val_loss: 0.3784 - val_accuracy: 0.8699 - 1s/epoch - 6ms/step
Epoch 7/200
232/232 - 1s - loss: 0.2974 - accuracy: 0.9020 - val_loss: 0.3598 - val_accuracy: 0.8690 - 1s/epoch - 6ms/step
Epoch 8/200
232/232 - 1s - loss: 0.2774 - accuracy: 0.9091 - val_loss: 0.3558 - val_accuracy: 0.8746 - 1s/epoch - 6ms/step
Epoch 9/200
232

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   10.7s


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.0s


Test time: 2.5741384029388428
Starting Naive Bayes
Training time: 92.78470349311829
Test time: 0.31665635108947754
Starting Decision tree
Training time: 94.01336932182312
Test time: 0.06181454658508301
Starting Multi layer perceptron
Iteration 1, loss = 2.97328868
Iteration 2, loss = 0.63466955
Iteration 3, loss = 0.61726814
Iteration 4, loss = 0.37697270
Iteration 5, loss = 0.71520041
Iteration 6, loss = 0.48683119
Iteration 7, loss = 0.62728789
Iteration 8, loss = 0.41089700
Iteration 9, loss = 0.31947532
Iteration 10, loss = 0.32963145
Iteration 11, loss = 0.59095642
Iteration 12, loss = 0.40874493
Iteration 13, loss = 0.37125176
Iteration 14, loss = 0.43176502
Iteration 15, loss = 0.52887703
Iteration 16, loss = 0.44351919
Iteration 17, loss = 0.34547609
Iteration 18, loss = 0.40683189
Iteration 19, loss = 0.49532166
Iteration 20, loss = 0.34098056
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Training time: 559.6124114990234
Test time: 0